In [ ]:
import sys
import spotipy
import spotipy.util as util
import os
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd 
import matplotlib.pyplot as plt
import openpyxl
from time import sleep

os.environ['SPOTIPY_CLIENT_ID'] = '15e2897ccbdf46a185534db50d4ad3e6'
os.environ['SPOTIPY_CLIENT_SECRET'] = '0cb03557b7e14dedad210d9cadff1d79'
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://google.com'


def get_playlist(username, playlist_send, playlist_receive, list_type):
    #Getting Playlist for analysis
    sourcePlaylist = sp.user_playlist(username, playlist_send)
    tracks = sourcePlaylist["tracks"]
    songs = tracks["items"] 
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks["items"]:
            songs.append(item) 
    for i in range(len(songs)):
        sp.user_playlist_remove_all_occurrences_of_tracks(username, playlist_receive, [songs[i]['track']['id']])

    ids = [] 
    print(len(songs))
    #print(songs[0]['track']['id'])
    i = 0
    for i in range(len(songs)):
        sp.user_playlist_add_tracks(username, playlist_receive, [songs[i]['track']['id']])

    #Getting Audio Features
    my_playlist = sp.user_playlist(username, playlist_send)
    my_tracks = my_playlist["tracks"]
    my_songs = my_tracks["items"]

    while my_tracks['next']:
        my_tracks = sp.next(my_tracks)
        for item in my_tracks["items"]:
            my_songs.append(item) 

    my_ids = []
    for i in range(len(my_songs)):
    
        my_ids.append(my_songs[i]['track']['id'])           

    features = []
    for i in range(0,len(my_ids),50):
        audio_features = sp.audio_features(my_ids[i:i+50])
        for track in audio_features:
            features.append(track)
            features[-1][list_type] = 1    
    return features



In [ ]:
# GOOD PLAYLIST

# Making two playlists to set up Classification Algorithm --> good and bad playlist, need playlist id for each, and playlist id of empty playlist where songs move to for analysis
playlist_types = {'good': ['0uPZV03oi77QuhufhTjvDQ', '6LLavZqMgsKCWUEuErceV8']}
export = 1

### AUTHENTICATION ###
cid = "15e2897ccbdf46a185534db50d4ad3e6"
secret = "0cb03557b7e14dedad210d9cadff1d79"
username = "126555631"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-modify-private playlist-read-private'
token = util.prompt_for_user_token(username, scope)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)
    
#######################

for play in playlist_types:
    print(play)
    ## GET DATA
    playlist = get_playlist(username, playlist_send = playlist_types.get(play)[0], playlist_receive = playlist_types.get(play)[1], list_type='target')       
    playlist = pd.DataFrame(playlist)
    
    ## EXPORT
    if export == 1:
        playlist.to_excel('~/data/spotify/{}_playlist.xlsx'.format(play))
        playlist.to_stata('~/data/spotify/{}_playlist.dta'.format(play))
        playlist.to_csv('~/data/spotify/{}_playlist.csv'.format(play))
    else:
        pass

In [ ]:
# BAD PLAYLIST
playlist_types = {'bad' : ['2eSTLF93PYMB5QMiTKB6HE', '7dVZB0451jPSVGPrgLiJkN']}
export = 1

### AUTHENTICATION ###
cid = "15e2897ccbdf46a185534db50d4ad3e6"
secret = "0cb03557b7e14dedad210d9cadff1d79"
username = "126555631"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-modify-private playlist-read-private'
token = util.prompt_for_user_token(username, scope)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)
    
#######################

for play in playlist_types:
    print(play)
    ## GET DATA
    bad_playlist = get_playlist(username, playlist_send = playlist_types.get(play)[0], playlist_receive = playlist_types.get(play)[1], list_type='no')       
    bad_playlist = pd.DataFrame(bad_playlist)
    
    ## EXPORT
    if export == 1:
        bad_playlist.to_excel('~/data/spotify/{}_playlist.xlsx'.format(play))
        bad_playlist.to_stata('~/data/spotify/{}_playlist.dta'.format(play))
        bad_playlist.to_csv('~/data/spotify/{}_playlist.csv'.format(play))
    else:
        pass

In [ ]:
#DATA ANALYSIS AND CLEAN
## using pandas
import pandas as pd

gplist = pd.read_csv('/Users/andrewsegers/data/spotify/good_playlist.csv')
bplist = pd.read_csv('/Users/andrewsegers/data/spotify/bad_playlist.csv')

bplist['target'] = 0
plist = [gplist, bplist]

plist = pd.concat(plist)

gfeatures = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
bfeatures = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']


In [ ]:
type(bplist)
# MATPLOTLIB & VISUALIZATION
## using plt.pyplot()

gfeatures.hist()
bfeatures.hist()

In [ ]:
x = [gfeatures, bfeatures]
x = pd.DataFrame(x)
y = plist['target']

plist = pd.DataFrame(plist)

In [ ]:
# SETUP TRAINING AND TESTING DATA
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
train, test = train_test_split(plist, test_size = 0.25)

In [ ]:
#Define the set of features that we want to look at
features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness"]
#Split the data into x and y test and train sets to feed them into a bunch of classifiers!
x_train = train[features]
y_train = train["target"]
x_test = test[features]
y_test = test["target"]

In [ ]:
c = DecisionTreeClassifier()
dt = c.fit(x_train, y_train)
y_pred = c.predict(x_test)
score = accuracy_score(y_test, y_pred) * 100
print("Accuracy using Decision Tree: ", round(score, 1), "%")


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(3)
knn.fit(x_train, y_train)
knn_pred = c.predict(x_test)
score = accuracy_score(y_test, knn_pred) * 100
print("Accuracy using Knn Tree: ", round(score, 1), "%")

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=100)
ada.fit(x_train, y_train)
ada_pred = ada.predict(x_test)

score = accuracy_score(y_test, ada_pred) * 100
print("Accuracy using ada: ", round(score, 1), "%")

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=0)
gbc.fit(x_train, y_train)
predicted = gbc.predict(x_test)

score = accuracy_score(y_test, predicted)*100
print("Accuracy using gbc: ", round(score, 1), "%")

In [ ]:
## GETTING PREDICTION PLAYLIST --> "i.e. 4 WEEKS OF DISCOVER WEEKLY PLAYLISTS"    
    
def get_disco_playlist(username, disco_plist, list_type):
    
    
    '''
    - Make private playlist of songs to test model on (i.e. Discover Weekly songs)
    - Getting Playlist for predictions
    - Run predictions:
        - How many did I like that the model said I would like? That I didn't like, but actually did? (false negative)
            - How many did it say I'd like but I didn't (false positive)?
    '''
    
    pred_plist = sp.user_playlist(username, disco_plist)
    tracks = pred_plist["tracks"]
    songs = tracks["items"] 
    while tracks['next']:
        tracks = sp.next(tracks)
        for item in tracks["items"]:
            songs.append(item)

    #Getting Audio Features
    my_playlist = sp.user_playlist(username, disco_plist)
    my_tracks = my_playlist["tracks"]
    my_songs = my_tracks["items"]

    while my_tracks['next']:
        my_tracks = sp.next(my_tracks)
        for item in my_tracks["items"]:
            my_songs.append(item) 

    my_ids = []
    for i in range(len(my_songs)):
    
        my_ids.append(my_songs[i]['track']['id'])           

    features = []
    for i in range(0,len(my_ids),50):
        audio_features = sp.audio_features(my_ids[i:i+50])
        for track in audio_features:
            features.append(track)
            #features[-1][list_type] = 1    
    return features
            


In [ ]:
### AUTHENTICATION ###
cid = "15e2897ccbdf46a185534db50d4ad3e6"
secret = "0cb03557b7e14dedad210d9cadff1d79"
username = "126555631"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-modify-private playlist-read-private'
token = util.prompt_for_user_token(username, scope)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)
    
#######################

## GET DATA AND PREP --> call function

disco_plist = get_disco_playlist(username, disco_plist = '3QF9KH9wIj0O0zwaFCoPMJ', list_type = 'prediction')
disco_plist = pd.DataFrame(disco_plist)

disco_plist.head()


In [ ]:
## PREDICTIONS

#pred = ada.predict(disco_plist[features])
pred = gbc.predict(disco_plist[features])
i = 0
count = 0
for prediction in pred:
    if(prediction == 1):
        count +=1
        print ("Song: " + disco_plist["uri"][i] + ", By: "+ disco_plist["track_href"][i])
    i = i +1
    
print(count)